In [2]:
from google import genai
import json
from typing import Generator, Iterator


In [3]:
with open(".env", "r") as f:
    API_KEY = f.readline()
MODEL = "gemini-2.5-flash-lite"

client = genai.Client(api_key=API_KEY)

def gemini(text: str) -> genai.types.GenerateContentResponse:
    return client.models.generate_content(model=MODEL, contents=text)

In [4]:
with open("longbench.json", "r", encoding="utf-8") as f:
    longbench = json.load(f)

In [5]:
def allWithQuality(kvPairs: list[tuple[str, str]]) -> Generator:
    for item in longbench:
        if all(item[k] == v for k,v in kvPairs):
            yield item

In [6]:
# response = gemini("What would you make the 27th letter of the alphabet")

In [10]:
for bench in allWithQuality([("difficulty", "easy"), ("length", "long")]):
    print(bench["question"])
    with open("out.txt", "w", encoding="utf-8") as f:
        f.write(bench["context"])
    break

In the function that calculates the derivative of given functions, which of the following keyword arguments are all recognized?


In [65]:
CHAR_TO_TOKEN_RATIO = .25
EPSILON = 1.1
TOKEN_LIMIT = 256

# str.split() except it makes sure we don't make any tiny splits
def splitMinLength(s: str, splitChar: str, minLength: int) -> list[str]:
    i = 0
    ret = []
    while i < len(s):
        item = ""
        while i < len(s):
            item += s[i]
            if item.endswith(splitChar) and len(item) >= minLength: 
                ret.append(item)
                break
            i += 1
        i += 1
    return ret

def chunkify(context: str, splitChar: str, minLength: int) -> Iterator[str]:    
    i = 0
    # splitContext = splitMinLength(context, splitChar, minLength)
    splitContext = [i + splitChar for i in context.split(splitChar)]
    while i < len(splitContext):
        chunk = ""
        collected = 0
        while i < len(splitContext) \
        and (len(splitContext[i]) * CHAR_TO_TOKEN_RATIO + collected) * EPSILON < TOKEN_LIMIT:
            chunk += splitContext[i]
            collected += (len(splitContext[i]) * CHAR_TO_TOKEN_RATIO)
            i += 1

        # If single chunk is still too big with current strategy fall back to simpler strat
        if chunk == "" and i < len(splitContext):
            if splitChar != "\n":
                yield from chunkify_lineBreaks(splitContext[i])
            else:
                yield from chunkify_naive(splitContext[i])
            i += 1
        elif chunk != "":
            yield chunk

def chunkify_lineBreaks(context: str) -> Iterator[str]:
    yield from chunkify(context, "\n", 0)

def chunkify_periods(context: str) -> Iterator[str]:
    yield from chunkify(context, ".", 50)

def chunkify_blankLines(context: str) -> Iterator[str]:
    yield from chunkify(context, "\n\n", 200)

def chunkify_naive(context: str) -> Iterator[str]:
    i = 0
    charsToSkip = int(TOKEN_LIMIT / CHAR_TO_TOKEN_RATIO / EPSILON)
    while i + charsToSkip < len(context):
        yield context[i:i+charsToSkip]
        i += charsToSkip
    yield context[i:]


In [68]:
testbench = longbench[0]
# print(testbench["question"])
with open("out.txt", "w", encoding="utf-8") as f:
    i = 0
    for chunk in chunkify_lineBreaks(testbench["context"]):
        f.write(f"-----Chunk {i+1}-----\n{chunk}\n")
        i += 1
        if i >= 10: break

In [73]:
response = None
for bench in allWithQuality([("difficulty", "easy"), ("length", "short")]):
    # print(bench["question"])
    # print("".join((f"{c}: {bench[f"choice_{c}"]}\n" for c in ["A", "B", "C", "D"])))
    # print(bench["answer"])
    prompt = f"You will be provided long context, and then a multiple choice question. Please attempt to choose the correct answer:\n"
    prompt += f"CONTEXT: {bench["context"]}\n"
    prompt += f"QUESTION: {bench["question"]}\n"
    prompt += ("".join((f"{c}: {bench[f"choice_{c}"]}\n" for c in ["A", "B", "C", "D"])))
    response = gemini(prompt)
    print(response.text)
    break



The question asks how to promote the construction of smart courts. Let's examine the provided text for information related to "smart courts" or "intelligent courts."

Section XI. "Advancing the Construction of Intelligent Courts" directly addresses this topic.
The introductory paragraph states: "Since 2013, the people’s courts have conscientiously implemented the innovation-driven strategy, the national cyber development strategy, the big data strategy, and the new-generation artificial intelligence development plan, and comprehensively strengthened the construction of intelligent courts. The open, and intelligent online Apps have been comprehensively developed, and the main framework of court informatization version 3.0 has been established, which greatly promotes the modernization of the judicial system and judicial capability."

Let's analyze the options:

*   **A: Through technology empowerment, change the way of working and improve office efficiency.** This aligns with the general

In [37]:
with open("out.txt", "w", encoding="utf-8") as f:
    f.write(testbench["context"])